# Practice: Strategy for find *ori*
- Textbook에서는 *ori* finding을 위한 흐름이 다음과 같았다.
    1. Whole genome 수준: 빈번한 단어 찾기 - 역상보 문제 - 패턴 일치 문제
    2. Local frequency: 군집 찾기 - 복제 비대칭성
    3. 대략적인 패턴 문제 (동일하지는 않지만 유사한 pattern)
- 그러나, 실제로 *ori* 를 찾기 위해서, 위와 같은 순서대로 진행하면 search space가 처음부터 너무 넓다.  
- 따라서, 다음과 같은 순서로 *ori* 를 찾고자 한다.  
    1. 복제 비대칭성: Whole genome 상에서 찾아야 할 부분을 거시적으로 탐색
    2. 군집 찾기: 일정한 크기의 window로 pattern 등장 빈도가 밀집된 영역 확인 - 1에서 찾은 부분이 들어맞는지 확인
    3. 각 Local area에서 빈번한지 확인

## Preparation for Practice
- Import packages
- Read file

In [12]:
# Basic os packages
import sys
import os

# Import local functions
dir_path = os.path.dirname(os.path.abspath('__file__'))
sys.path.append(dir_path)

# Import external packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
# Read file
path = "./Vibrio_cholerae.txt"
f = open(path, 'r')
genome = f.readline().strip()

## G-C ratio Assymmetry
### Goals
 *ori* 가 있을법한 후보 "영역" 찾기

### Steps
1. 임의의 개수로 bin을 나누어 각 bin 별로 base 빈도가 얼마나 되는지 확인
2. Assymmetry diagram을 통해 각 minima, maxima 확인

In [3]:
print(len(genome))

1108250


In [49]:
bins = []
binNumber = 12
genomeSize = len(genome)

for bin in range(binNumber):
    start = genomeSize * bin // binNumber
    end = genomeSize * (bin + 1) // binNumber
    bins.append(genome[start:end])

In [146]:
binNumber = 12
genomeSize = len(genome)
binSize = genomeSize // binNumber + 1

bins = np.empty((binSize, binNumber), dtype=np.object_)

# genome을 binNumber로 나누어서 bins에 저장
for nbin in range(binNumber):
    start = genomeSize * nbin // binNumber
    end = genomeSize * (nbin + 1) // binNumber
    
    binContents = genome[start:end]
    
    for idx in range(len(binContents)):
        bins[idx][nbin] = binContents[idx]
    
    # binSize보다 작으면 N으로 채움
    if len(binContents) != binSize: 
        bins[-1][nbin] = 'N'

In [147]:
print(bins, bins.shape)

[['A' 'G' 'T' ... 'A' 'T' 'C']
 ['C' 'C' 'T' ... 'T' 'T' 'G']
 ['A' 'A' 'G' ... 'C' 'T' 'G']
 ...
 ['T' 'G' 'G' ... 'G' 'C' 'A']
 ['G' 'A' 'C' ... 'T' 'T' 'A']
 ['N' 'N' 'N' ... 'N' 'N' 'A']] (92355, 12)


In [152]:
np.unique(bins[:], return_counts=True)

(array(['A', 'C', 'G', 'N', 'T'], dtype=object),
 array([293942, 263573, 256024,     10, 294711]))

In [153]:
binDF = pd.DataFrame(bins)
binDF.describe()

,0,1,2,3,4,5,6,7,8,9,10,11
count,92355,92355,92355,92355,92355,92355,92355,92355,92355,92355,92355,92355
unique,5,5,5,5,5,4,5,5,5,5,5,4
top,A,A,T,T,T,T,A,T,A,A,A,T
freq,24268,24363,24605,24587,25049,24992,24355,24431,26063,28050,24430,24656
